In [1]:
from greedy_mcmc_attack import *
from data_collection.recordings import *
import sys
sys.path.append("../../")
from src.models.gcn import *
from src.models.gat import *
from src.models.gsage import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

trials = 5

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
edges_to_add = None

In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [6]:
model_save_path = "../models/cora_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges_full_run.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [7]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [8]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.806
Initial Loss: 0.655889093875885


In [9]:
G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.20)

In [ ]:
full_metattack_acc, full_metattack_loss, full_metattack_itrs = [], [], []
for _ in range(trials):
    acc, loss, itrs = two_phase_attack_mcmc(data, train, model, 0.0, edges_to_add, binary_fn, constant_fn, device, is_reversed=False, verbose=True)
    full_metattack_acc.append(acc)
    full_metattack_loss.append(loss)
    full_metattack_itrs.append(itrs)

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6236100792884827
Surrogate acc on unlabled data: 0.798
Attack loss: 0.28566595911979675
Surrogate loss on unlabled data: 0.6994355320930481
Surrogate acc on unlabled data: 0.782
Attack loss: 0.4052053689956665
Surrogate loss on unlabled data: 0.8351637125015259
Surrogate acc on unlabled data: 0.755
Attack loss: 0.5723212957382202
Surrogate loss on unlabled data: 0.8673900365829468
Surrogate acc on unlabled data: 0.743
Attack loss: 0.6142011880874634
Surrogate loss on unlabled data: 0.9535175561904907
Surrogate acc on unlabled data: 0.714
Attack loss: 0.7186201214790344
Surrogate loss on unlabled data: 1.054160714149475
Surrogate acc on unlabled data: 0.7000000000000001
Attack loss: 0.8014763593673706
Surrogate loss on unlabled data: 1.1016607284545898
Surrogate acc on unlabled data: 0.681
Attack loss: 0.8752995133399963
Surrogate loss on unlabled data: 1.2500213384628296
Surrogate acc on unlabled data: 0.668
Attack loss: 1.0125107765197754
Surrogate l

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6105505228042603
Surrogate acc on unlabled data: 0.796
Attack loss: 0.28948280215263367
Surrogate loss on unlabled data: 0.7543007731437683
Surrogate acc on unlabled data: 0.777
Attack loss: 0.4499451518058777
Surrogate loss on unlabled data: 0.8335283994674683
Surrogate acc on unlabled data: 0.745
Attack loss: 0.5339338779449463
Surrogate loss on unlabled data: 0.9373815655708313
Surrogate acc on unlabled data: 0.737
Attack loss: 0.6194344758987427
Surrogate loss on unlabled data: 0.9992719292640686
Surrogate acc on unlabled data: 0.715
Attack loss: 0.7294621467590332
Surrogate loss on unlabled data: 1.1124342679977417
Surrogate acc on unlabled data: 0.6910000000000001
Attack loss: 0.8369702696800232
Surrogate loss on unlabled data: 1.2044308185577393
Surrogate acc on unlabled data: 0.673
Attack loss: 0.9409142136573792
Surrogate loss on unlabled data: 1.2444406747817993
Surrogate acc on unlabled data: 0.647
Attack loss: 0.9901265501976013
Surrogate 

# regular, secondary attack, not reversed

In [ ]:
splits = [0.0, 0.5]
split_dic_acc = defaultdict(lambda: defaultdict(list))
split_dic_loss = defaultdict(lambda: defaultdict(list))
itrs_one = defaultdict(lambda: defaultdict(int))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = two_phase_attack_mcmc(data, train, model, s, edges_to_add, binary_fn, constant_fn, device, is_reversed=False, verbose=True)

In [ ]:
record1 = Recording(split_dic_loss, split_dic_acc, itrs_one, Model.GCN, Dataset.CORA, AcceptFn.CONSTANT, SelectFn.BINARY, Reverse.ATTACKED)
record1.save(f'data_collection/{"mcmc"}_{"gcn"}_{"cora"}_{"constant"}_{"binary"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_acc, ptb_rate, "MCMC", "binary", "constant", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_loss, ptb_rate, "MCMC", "binary", "constant", "loss", "regular")

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

In [ ]:
split_dic_decreasing_acc = defaultdict(lambda: defaultdict(list))
itrs_two = defaultdict(lambda: defaultdict(int))
split_dic_decreasing_loss = defaultdict(lambda: defaultdict(list))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = two_phase_attack_mcmc(data, train, model, s, edges_to_add, binary_fn, increasing_fn, device, is_reversed=False, verbose=True)

In [ ]:
record1 = Recording(split_dic_decreasing_loss, split_dic_decreasing_acc, itrs_two, Model.GCN, Dataset.CORA, AcceptFn.INCREASING, SelectFn.BINARY, Reverse.ATTACKED)
record1.save(f'data_collection/{"mcmc"}_{"gcn"}_{"cora"}_{"increasing"}_{"binary"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_acc, ptb_rate, "MCMC", "binary", "increasing", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_loss, ptb_rate, "MCMC", "binary", "increasing", "loss", "regular")

In [ ]:
split_dic_decreasing_constant_acc = defaultdict(lambda: defaultdict(list))
split_dic_decreasing_constant_loss = defaultdict(lambda: defaultdict(list))
itrs_three = defaultdict(lambda: defaultdict(int))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_constant_acc[s][j], split_dic_decreasing_constant_loss[s][j], itrs_three[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_constant_acc[s][j], split_dic_decreasing_constant_loss[s][j], itrs_three[s][j] = two_phase_attack_mcmc(data, train, model, s, edges_to_add, decreasing_fn, constant_fn, device, is_reversed=False, verbose=True)

In [ ]:
record1 = Recording(split_dic_decreasing_constant_loss, split_dic_decreasing_constant_acc, itrs_three, Model.GCN, Dataset.CORA, AcceptFn.CONSTANT, SelectFn.DECAYING, Reverse.ATTACKED)
record1.save(f'data_collection/{"mcmc"}_{"gcn"}_{"cora"}_{"constant"}_{"decaying"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_constant_acc, ptb_rate, "MCMC", "decaying", "constant", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_constant_loss, ptb_rate, "MCMC", "decaying", "constant", "loss", "regular")

In [ ]:
split_dic_decreasing_gradual_acc = defaultdict(lambda: defaultdict(list))
split_dic_decreasing_gradual_loss = defaultdict(lambda: defaultdict(list))
itrs_four = defaultdict(lambda: defaultdict(int))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_gradual_acc[s][j], split_dic_decreasing_gradual_loss[s][j], itrs_four[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_gradual_acc[s][j], split_dic_decreasing_gradual_loss[s][j], itrs_four[s][j] = two_phase_attack_mcmc(data, train, model, s, edges_to_add, decreasing_fn, increasing_fn, device, is_reversed=False, verbose=True)

In [ ]:
record1 = Recording(split_dic_decreasing_gradual_loss, split_dic_decreasing_gradual_acc, itrs_four, Model.GCN, Dataset.CORA, AcceptFn.CONSTANT, SelectFn.DECAYING, Reverse.ATTACKED)
record1.save(f'data_collection/{"mcmc"}_{"gcn"}_{"cora"}_{"increasing"}_{"decaying"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_gradual_acc, ptb_rate, "MCMC", "decaying", "increasing", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_gradual_loss, ptb_rate, "MCMC", "decaying", "increasing", "accuracy", "regular")